In [ ]:
import importlib
import src.ri.get_data.transaction_pipeline as tp 
importlib.reload(tp)
tp.create_weekly_transactions_master_table()

In [1]:
# JUPYTER CELL 0 — config & imports

PROJECT = "cart-dai-sandbox-nonprod-c3e7"
DATASET = "dlorino"

TX_TABLE = f"{PROJECT}.{DATASET}.sku_store_week_sales_base"          # built by create_weekly_transactions_master_table()
RANKED_TABLE = f"{PROJECT}.{DATASET}.sku_store_week_sales_ranked"   # built by create_weekly_transactions_master_table()
MEDIA_TABLE = f"{PROJECT}.{DATASET}.media_master_v1"                # built by your media SQL once

OUTPUT_DIR = "d_outputs_nb"   # notebook run outputs go here

# If you rely on ADC, make sure GOOGLE_APPLICATION_CREDENTIALS is set in the notebook kernel environment.

import os, pandas as pd
os.makedirs(OUTPUT_DIR, exist_ok=True)


## Create Transactions Master Table

In [ ]:
# JUPYTER CELL 1 — build/refresh transactions master (BQ)

from google.cloud import bigquery
# NOTE: adjust the import path if your file is placed differently:
from src.ri.get_data.transaction_pipeline import create_weekly_transactions_master_table

client = bigquery.Client(project=PROJECT)
_ = create_weekly_transactions_master_table(client=client)  # writes both *_pn2 and *_ranked
print("Transactions master refreshed.")


## Create Media Master Table 

In [8]:
# JUPYTER CELL 2 — build/refresh media master (BQ)

from google.cloud import bigquery
from pathlib import Path

# If you've saved the SQL at 0_precompute/sql/media_master.sql, use that:
sql_path = "x_precompute/sql/media_master.sql"

if os.path.exists(sql_path):
    sql = Path(sql_path).read_text()
else:
    # Fallback: paste your SQL string directly here if you haven't saved the file.
    raise FileNotFoundError("Put your media_master.sql at x_precompute/sql/media_master.sql or paste it here as a string.")

sql = (sql
       .replace("{{PROJECT}}", PROJECT)
       .replace("{{DATASET}}", DATASET)
       .replace("{{TABLE}}", "media_master_v1"))

client = bigquery.Client(project=PROJECT)
client.query(sql).result()
print(f"Media master written to {MEDIA_TABLE}")


/opt/homebrew/lib/python3.11/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Media master written to cart-dai-sandbox-nonprod-c3e7.dlorino.media_master_v1


## Load Media Campaigns

In [2]:
# JUPYTER CELL 3 — load media master and select campaigns

from google.cloud import bigquery
from src.ri.get_data.media.parse_media_table import to_list_flexible

client = bigquery.Client(project=PROJECT)

media_sql = f"""
SELECT booking_number, opportunity_name,
       campaign_start_date, campaign_end_date,
       media_start_date, media_end_date,
       media_array, media_location_array, media_type_array,
       sorted_store_list, sorted_sku_list,
       campaign_week, campaign_week_split
FROM `{MEDIA_TABLE}`
ORDER BY booking_number, media_start_date, media_end_date
"""
media_df = client.query(media_sql).result().to_dataframe()

# Coerce list-ish fields into Python lists (your utility can handle list or comma string)
for col in ["media_array","media_location_array","media_type_array","sorted_store_list","sorted_sku_list"]:
    media_df[col] = media_df[col].apply(to_list_flexible)

print(len(media_df), "media rows loaded")

# ---- A) ALL campaigns
bookings_all = sorted(media_df["booking_number"].astype(str).unique().tolist())
print("All campaigns:", len(bookings_all))

# ---- B) window mode (adjust dates)
START, END = "2024-02-01", "2024-02-14"
mask = (pd.to_datetime(media_df["campaign_start_date"]) >= pd.to_datetime(START)) & \
       (pd.to_datetime(media_df["campaign_end_date"])   <= pd.to_datetime(END))
bookings_window = sorted(media_df.loc[mask, "booking_number"].astype(str).unique().tolist())
print(f"Window campaigns ({START}..{END}):", len(bookings_window))


/opt/homebrew/lib/python3.11/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


17152 media rows loaded
All campaigns: 6611
Window campaigns (2024-02-01..2024-02-14): 5


In [3]:
bookings_window

['WOW20006579', 'WOW20006936', 'WOW20007663', 'WOW20008200', 'WOW20009671']

## Forecast Pipeline

In [26]:
# JUPYTER CELL 4 — run the pipeline (ALL or WINDOW)
# Choose one:
BOOKINGS = bookings_window      # run a window
# BOOKINGS = bookings_all       # or run everything

# Choose metrics and grains (you can pass ["all"] to use the defaults inside the runner)
METRICS = ["all"] #["sales","shoppers", "new_to_brand_sales","new_to_brand_shoppers"]
GRAINS  = ["brand_category","brand_subcategory","brand","category","subcategory","sku"]

# We import the function that the CLI script uses under the hood.
import importlib
import src.ri.model.forecasting.aggregate_nodes as agg
import src.ri.model.forecasting.cohort_forecasting as cf
import src.ri.model.forecasting.forecasters as fore
import x_scripts.run_from_media_table as media_module
import src.ri.get_data.media.build_weekly_store_calendar as calmod
importlib.reload(calmod)

importlib.reload(agg); importlib.reload(fore); importlib.reload(cf); importlib.reload(media_module)

# Subset media_df to the campaigns we want (this avoids extra work inside run_pipeline)
media_sel = media_df[media_df["booking_number"].astype(str).isin(set(BOOKINGS))].copy()

media_module.run_pipeline2(
    media=media_sel,
    tx_table=TX_TABLE,        # BQ live slicing (recommended)
    tx_parquet=None,          # leave as None when using tx_table
    project=PROJECT,
    outdir=OUTPUT_DIR,
    metrics=METRICS,
    grains=GRAINS
)

print("Pipeline run complete.")


run_pipeline2 setup... 

/opt/homebrew/lib/python3.11/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Done.


  0%|          | 0/5 [00:00<?, ?it/s]


booking_number WOW20006579
WOW20006579 Calendar: 
Empty DataFrame
Columns: [week_start, store_id, is_covered]
Index: []
wk_to_stores
{}
Campaign Start: 2024-02-07 00:00:00
Campaign End: 2024-02-13 00:00:00
Campaign Weeks: [Timestamp('2024-02-01 00:00:00'), Timestamp('2024-02-08 00:00:00')]
[Calendar] ALL STORES; synthesized 2 campaign weeks from media dates.
[Preflight] SKUs (14): ['189052', '335282', '335285', '335291', '352022', '352026', '352027', '396293', '59484', '662850', '662865', '682628', '682629', '792923']
[Preflight] Stores (ALL STORES): ~1017
[Preflight] Brands (5): ['BLACKHAWK', 'GREATER UNION', 'PROPRIETARY', 'RED BALLOON', 'VILLAGE THEME']
[Skip] booking WOW20006579: promo SKUs span 5 brands: ['BLACKHAWK', 'GREATER UNION', 'PROPRIETARY', 'RED BALLOON', 'VILLAGE THEME']

booking_number WOW20006936
WOW20006936 Calendar: 
    week_start store_id  is_covered
0   2024-02-01     1069           1
1   2024-02-01     1085           1
2   2024-02-01     1121           1
3   202

In [22]:
check = pd.read_parquet("d_outputs_nb/artifacts/WOW20007663/compare_shoppers_sku.parquet")
check

,ds,group,product_id,level,y,yhat


In [24]:
import pandas as pd, glob, os
booking = "WOW20007663"  # one that wasn't skipped
root = os.path.join(OUTPUT_DIR, "artifacts", booking)
for f in sorted(glob.glob(os.path.join(root, "compare_*_*.parquet"))):
    df = pd.read_parquet(f)
    print(os.path.basename(f), df.shape)
    print(df.head(3))

compare_new_to_brand_sales_sku.parquet (0, 6)
Empty DataFrame
Columns: [ds, group, product_id, level, y, yhat]
Index: []
compare_new_to_brand_shoppers_sku.parquet (0, 6)
Empty DataFrame
Columns: [ds, group, product_id, level, y, yhat]
Index: []
compare_new_to_sku_sales_sku.parquet (0, 6)
Empty DataFrame
Columns: [ds, group, product_id, level, y, yhat]
Index: []
compare_new_to_sku_shoppers_sku.parquet (0, 6)
Empty DataFrame
Columns: [ds, group, product_id, level, y, yhat]
Index: []
compare_sales_sku.parquet (0, 6)
Empty DataFrame
Columns: [ds, group, product_id, level, y, yhat]
Index: []
compare_shoppers_sku.parquet (0, 6)
Empty DataFrame
Columns: [ds, group, product_id, level, y, yhat]
Index: []
